In [1]:
# Work in progress, not a complete example.

import argparse
import gzip
import os
import sys
import time
import urllib

import numpy
import tensorflow as tf

from six.moves import xrange

In [2]:
# MNIST example (data downloading etc.) is based on Google's https://github.com/tensorflow/models/blob/master/tutorials/image/mnist/convolutional.py
# See LICENSE for details.

SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/'
WORK_DIRECTORY = 'data'
IMAGE_SIZE = 28
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 5000  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64
NUM_EPOCHS = 10
EVAL_BATCH_SIZE = 64
EVAL_FREQUENCY = 100  # Number of steps between evaluations.

EMBEDDING_SIZE = 10
NUM_INTERNAL_CONVS = 5
NUM_UNROLL_STEPS = 5

In [3]:
def maybe_download(filename):
  """Download the data from Yann's website, unless it's already here."""
  if not tf.gfile.Exists(WORK_DIRECTORY):
    tf.gfile.MakeDirs(WORK_DIRECTORY)
  filepath = os.path.join(WORK_DIRECTORY, filename)
  if not tf.gfile.Exists(filepath):
    filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
    with tf.gfile.GFile(filepath) as f:
      size = f.size()
    print('Successfully downloaded', filename, size, 'bytes.')
  return filepath


def extract_data(filename, num_images):
  """Extract the images into a 4D tensor [image index, y, x, channels].

  Values are rescaled from [0, 255] down to [-0.5, 0.5].
  """
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(16)
    buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
    data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32)
    data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
    data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
    return data


def extract_labels(filename, num_images):
  """Extract the labels into a vector of int64 label IDs."""
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    bytestream.read(8)
    buf = bytestream.read(1 * num_images)
    labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64)
  return labels

from collections import Counter

def error_rate_single(predictions, labels):
  """Return the error rate based on dense predictions and sparse labels."""
  argmax = numpy.argmax(predictions, 1)
  #print(Counter(argmax).most_common())
  return 100.0 - (
      100.0 *
      numpy.sum(argmax == labels) /
      predictions.shape[0])


def error_rate(predictions_all_steps, labels):
  return [error_rate_single(predictions_all_steps[i], labels) for i in range(NUM_UNROLL_STEPS)]

train_data_filename = maybe_download('train-images-idx3-ubyte.gz')
train_labels_filename = maybe_download('train-labels-idx1-ubyte.gz')
test_data_filename = maybe_download('t10k-images-idx3-ubyte.gz')
test_labels_filename = maybe_download('t10k-labels-idx1-ubyte.gz')

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, 60000)
train_labels = extract_labels(train_labels_filename, 60000)
test_data = extract_data(test_data_filename, 10000)
test_labels = extract_labels(test_labels_filename, 10000)

# Generate a validation set.
validation_data = train_data[:VALIDATION_SIZE, ...]
validation_labels = train_labels[:VALIDATION_SIZE]
train_data = train_data[VALIDATION_SIZE:, ...]
train_labels = train_labels[VALIDATION_SIZE:]
train_size = train_labels.shape[0]

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
def feature_extractor(input_images, training):
    x = input_images
    x = tf.layers.conv2d(x, filters=16, kernel_size=[5,5], strides=1,
                         padding="VALID")
    x = tf.layers.batch_normalization(x, momentum=0.9, scale=False, fused=True, training=training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters=32, kernel_size=[3,3], strides=1,
                         padding="VALID")
    x = tf.layers.batch_normalization(x, momentum=0.9, scale=False, fused=True, training=training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters=64, kernel_size=[3,3], strides=1,
                         padding="VALID")
    x = tf.layers.batch_normalization(x, momentum=0.9, scale=False, fused=True, training=training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters=128, kernel_size=[3,3], strides=1,
                         padding="VALID")
    x = tf.layers.batch_normalization(x, momentum=0.9, fused=True, training=training)
    x = tf.nn.relu(x)
    return x

def model_step(input_images, prior, batch_size):
    """The Model definition."""
    prior_embeddings = tf.get_variable("prior_embeddings",
                                        shape=[NUM_LABELS, EMBEDDING_SIZE - 1],
                                        initializer=tf.random_uniform_initializer(
                                            minval=-1.0/numpy.sqrt(NUM_LABELS), maxval=1.0/numpy.sqrt(NUM_LABELS)))
    raw_embedding_features = tf.matmul(prior, prior_embeddings)
    prior_entropy = - tf.reduce_sum(prior * tf.log(1e-4 + prior), axis=-1, keepdims=True)
    embedding_features = tf.concat([raw_embedding_features, prior_entropy], axis=-1)
    for i in range(4):
        gates = tf.layers.dense(
            embedding_features, EMBEDDING_SIZE, activation=tf.nn.sigmoid)
        embedding_features = gates * (embedding_features + tf.layers.dense(
            embedding_features, EMBEDDING_SIZE, use_bias=False, activation=tf.nn.relu))
    
    def get_dynamic_weights(weights_shape):
        num_weights = numpy.prod(weights_shape[1:])
        dynamic_weights_flat = tf.layers.dense(embedding_features, num_weights)
        dynamic_weights = tf.reshape(dynamic_weights_flat, weights_shape)
        dynamic_weights.set_shape(weights_shape)
        return dynamic_weights
    
    conv1_weights = get_dynamic_weights([batch_size, 1, 1, 128, EMBEDDING_SIZE])
    conv2_weights = [get_dynamic_weights([batch_size, 2, 2, EMBEDDING_SIZE, EMBEDDING_SIZE])
                     for _ in range(NUM_INTERNAL_CONVS)]
    conv3_weights = get_dynamic_weights([batch_size, 1, 1, EMBEDDING_SIZE, NUM_LABELS])
    conv3_biases = get_dynamic_weights([batch_size, NUM_LABELS])
    
    logits_lst, posteriors_lst = [], []
    for elm in range(batch_size):
        conv = tf.nn.conv2d([input_images[elm]],
                            conv1_weights[elm],
                            strides=[1, 1, 1, 1],
                            padding='VALID')
        relu = tf.nn.relu(conv)
        for i in range(NUM_INTERNAL_CONVS):
            conv = tf.nn.conv2d(relu,
                            conv2_weights[i][elm],
                            strides=[1, 1, 1, 1],
                            padding='SAME')
            relu = tf.nn.relu(conv) + relu
        conv = tf.nn.conv2d(relu,
                            conv3_weights[elm],
                            strides=[1, 1, 1, 1],
                            padding='VALID')
        conv_shape = conv.get_shape()
        logits = tf.nn.avg_pool(conv, ksize=[1, conv_shape[1], conv_shape[2], 1],
                                strides=[1, 1, 1, 1], padding="VALID") + conv3_biases[elm]
        logits = tf.squeeze(logits, axis=[1, 2])
        posteriors = tf.nn.softmax(logits)
        logits_lst.append(logits)
        posteriors_lst.append(posteriors)
    return tf.concat(logits_lst, axis=0), tf.concat(posteriors_lst, axis=0)

def apply(input_images, training):
    results = []
    loss = 0.0
    conv_features = feature_extractor(input_images, training=training)
    batch_size = conv_features.get_shape()[0]  # HyperNet operates on single images only
    priors = numpy.array([[1/NUM_LABELS for _ in range(NUM_LABELS)] for _ in range(batch_size)],
                         dtype=numpy.float32)
    for step in range(NUM_UNROLL_STEPS):
        logits, posteriors = model_step(conv_features, priors, batch_size)
        priors = posteriors
        results.append((logits, posteriors))
        loss += tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=train_labels_node, logits=logits))
    return tf.stack([logits for (logits, _) in results]), loss

In [5]:
tf.reset_default_graph()

train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
train_labels_node = tf.placeholder(tf.int64, shape=(BATCH_SIZE,))
eval_data = tf.placeholder(
    tf.float32,
    shape=(EVAL_BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))


# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0, dtype=tf.float32)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    1e-3,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)

# Predictions for the current training minibatch.
with tf.variable_scope("model", reuse=tf.AUTO_REUSE):
    train_prediction, loss = apply(train_data_node, training=True)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,
                                                   global_step=batch)


# Predictions for the test and validation, which we'll compute less often.
with tf.variable_scope("model", reuse=True):
    eval_prediction, _ = apply(eval_data, training=False)
  # Small utility function to evaluate a dataset by feeding batches of data to
  # {eval_data} and pulling the results from {eval_predictions}.
  # Saves memory and enables this to run on smaller GPUs.
def eval_in_batches(data, sess):
    """Get all predictions for a dataset by running it in small batches."""
    size = data.shape[0]
    if size < EVAL_BATCH_SIZE:
        raise ValueError("batch size for evals larger than dataset: %d" % size)
    predictions = numpy.ndarray(shape=(NUM_UNROLL_STEPS, size, NUM_LABELS), dtype=numpy.float32)
    for begin in xrange(0, size, EVAL_BATCH_SIZE):
        end = begin + EVAL_BATCH_SIZE
        if end <= size:
            predictions[:, begin:end, :] = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[begin:end, ...]})
        else:
            batch_predictions = sess.run(
                eval_prediction,
                feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]})
            predictions[:, begin:, :] = batch_predictions[:, begin - size:, :]
    return predictions

  # Create a local session to run the training.
start_time = time.time()
with tf.Session() as sess:
    # Run all the initializers to prepare the trainable parameters.
    tf.global_variables_initializer().run()
    print('Initialized!')
    # Loop through training steps.
    for step in xrange(int(NUM_EPOCHS * train_size) // BATCH_SIZE):
      # Compute the offset of the current minibatch in the data.
      # Note that we could use better randomization across epochs.
      offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
      batch_data = train_data[offset:(offset + BATCH_SIZE), ...]
      batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
      # This dictionary maps the batch data (as a numpy array) to the
      # node in the graph it should be fed to.
      feed_dict = {train_data_node: batch_data,
                   train_labels_node: batch_labels}
      # Run the optimizer to update weights.
      sess.run(optimizer, feed_dict=feed_dict)
      # print some extra information once reach the evaluation frequency
      if step % EVAL_FREQUENCY == 0:
        # fetch some extra nodes' data
        l, lr, predictions = sess.run([loss, learning_rate, train_prediction],
                                      feed_dict=feed_dict)
        elapsed_time = time.time() - start_time
        start_time = time.time()
        print('Step %d (epoch %.2f), %.1f ms' %
              (step, float(step) * BATCH_SIZE / train_size,
               1000 * elapsed_time / EVAL_FREQUENCY))
        print('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
        print('Minibatch error: {}'.format(error_rate(predictions, batch_labels)))
        print('Validation error: {}'.format(error_rate(
            eval_in_batches(validation_data, sess), validation_labels)))
        sys.stdout.flush()
    # Finally print the result!
    test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
    print('Test error: {}'.format(test_error))


Initialized!
Step 0 (epoch 0.00), 326.2 ms
Minibatch loss: 11.597, learning rate: 0.001000
Minibatch error: [89.0625, 87.5, 92.1875, 85.9375, 89.0625]
Validation error: [89.98, 89.0, 91.32, 90.76, 89.98]
Step 100 (epoch 0.12), 551.0 ms
Minibatch loss: 5.422, learning rate: 0.001000
Minibatch error: [50.0, 53.125, 35.9375, 21.875, 21.875]
Validation error: [60.76, 58.42, 47.6, 27.060000000000002, 31.599999999999994]
Step 200 (epoch 0.23), 534.2 ms
Minibatch loss: 4.337, learning rate: 0.001000
Minibatch error: [48.4375, 20.3125, 31.25, 14.0625, 20.3125]
Validation error: [52.14, 29.480000000000004, 38.92, 29.36, 35.0]
Step 300 (epoch 0.35), 504.0 ms
Minibatch loss: 4.021, learning rate: 0.001000
Minibatch error: [26.5625, 9.375, 15.625, 18.75, 17.1875]
Validation error: [25.120000000000005, 17.060000000000002, 20.379999999999995, 17.379999999999995, 16.680000000000007]
Step 400 (epoch 0.47), 533.3 ms
Minibatch loss: 2.896, learning rate: 0.001000
Minibatch error: [21.875, 10.9375, 12.5,

Step 3400 (epoch 3.96), 529.4 ms
Minibatch loss: 0.265, learning rate: 0.000857
Minibatch error: [3.125, 1.5625, 1.5625, 1.5625, 1.5625]
Validation error: [4.060000000000002, 2.8400000000000034, 2.5799999999999983, 2.280000000000001, 2.0]
Step 3500 (epoch 4.07), 515.1 ms
Minibatch loss: 0.392, learning rate: 0.000815
Minibatch error: [1.5625, 3.125, 3.125, 1.5625, 0.0]
Validation error: [3.0799999999999983, 2.299999999999997, 2.0799999999999983, 1.7999999999999972, 1.9599999999999937]
Step 3600 (epoch 4.19), 498.4 ms
Minibatch loss: 0.152, learning rate: 0.000815
Minibatch error: [0.0, 1.5625, 0.0, 1.5625, 0.0]
Validation error: [2.819999999999993, 2.219999999999999, 2.239999999999995, 2.239999999999995, 1.980000000000004]
Step 3700 (epoch 4.31), 517.6 ms
Minibatch loss: 0.051, learning rate: 0.000815
Minibatch error: [0.0, 0.0, 0.0, 0.0, 0.0]
Validation error: [3.019999999999996, 2.0, 1.980000000000004, 1.7600000000000051, 1.7199999999999989]
Step 3800 (epoch 4.42), 507.8 ms
Minibatch

Validation error: [1.9599999999999937, 1.8599999999999994, 1.6400000000000006, 1.2399999999999949, 1.2199999999999989]
Step 6800 (epoch 7.91), 503.7 ms
Minibatch loss: 0.045, learning rate: 0.000698
Minibatch error: [1.5625, 0.0, 0.0, 0.0, 0.0]
Validation error: [2.180000000000007, 1.4599999999999937, 1.7999999999999972, 1.3599999999999994, 1.3400000000000034]
Step 6900 (epoch 8.03), 525.4 ms
Minibatch loss: 0.173, learning rate: 0.000663
Minibatch error: [1.5625, 3.125, 0.0, 0.0, 0.0]
Validation error: [2.280000000000001, 1.7999999999999972, 1.5799999999999983, 1.480000000000004, 1.3599999999999994]
Step 7000 (epoch 8.15), 503.5 ms
Minibatch loss: 0.243, learning rate: 0.000663
Minibatch error: [3.125, 1.5625, 1.5625, 0.0, 0.0]
Validation error: [2.719999999999999, 1.8199999999999932, 1.3400000000000034, 1.4599999999999937, 1.5799999999999983]
Step 7100 (epoch 8.26), 517.6 ms
Minibatch loss: 0.477, learning rate: 0.000663
Minibatch error: [3.125, 1.5625, 1.5625, 1.5625, 1.5625]
Valida